<a href="https://colab.research.google.com/github/baichlseder/voila/blob/main/voila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U fastai 

     |████████████████████████████████| 194kB 4.2MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 12.8MB 37.9MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [2]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [3]:
def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [4]:
file_id = 'OujMPsJ5fPGSaCkoHQiJKiemcMyzOyK'
destination = './handwritting_classifier.pkl'
download_file_from_google_drive(file_id, destination)

In [5]:
!cp /content/drive/MyDrive/handwritting_classifier.pkl ./sample_data

## Classifier zu Nummern
Es werden alle Zahlen von 0 bis 9 erkannt:
  * 0
  * 1
  * 2
  * 3
  * 4
  * 5
  * 6
  * 7
  * 8
  * 9

In [6]:
path = Path()
inference = load_learner('./sample_data/handwritting_classifier.pkl')

In [7]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl : display(img.to_thumb(128,128))
  pred, pred_idx, probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your number!'),
              btn_upload, btn_run, out_pl, lbl_pred])